In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

## 1. URL von allen Hauptseiten holen

In [2]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [3]:
pd.DataFrame(alle_seiten)
df_a=pd.DataFrame(alle_seiten)
# Die Geschäftsnummer zur Zahl machen. 
df_a["VorlageNR"] = df_a["VorlageNR"].astype(int) 

## 1.1 Selektion: Fokus auf Vorstösse
So will ich eine allzu grosse pdf-Schwemme verhindern.
Die Idee: nur jene Geschäfte auf den PC ziehen, die auch relevant sind für die Auswertung. 
Also nur Vorstösse. Mich so bis zu den PDF's vorkämpfen, um auch hier nur jene auf meinem Rechner zu haben, 
die ich wirklich auf meinem Rechner will.

In [4]:
#hier mache ich eine Liste, die nur aus denjenigen URLs besteht, die Vorstösse sind. 

list_of_vorstoss=["Motion","Interpellation","Postulat","Kleine Anfrage", "Initiative"]
df_v=df_a[df_a["Art des Geschäfts"].isin (list_of_vorstoss)]

In [5]:
df_v #hier habe ich nun alle Vorstösse

,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,3031,/gast/geschaefte/2018,Interpellation der SVP-Fraktion betreffend Sic...,Einreichung,Direktion für Bildung und Kultur,Interpellation,hängig
1,3030,/gast/geschaefte/2017,Motion der SVP-Fraktion betreffend Stärkung de...,Einreichung,,Motion,hängig
2,3029,/gast/geschaefte/2016,Interpellation der Fraktion Alternative - die ...,Einreichung,Volkswirtschaftsdirektion,Interpellation,hängig
5,3026,/gast/geschaefte/2013,Interpellation von Markus Spörri und Thomas Ga...,Einreichung,Baudirektion,Interpellation,hängig
7,3024,/gast/geschaefte/2012,Interpellation von Claus Soltermann und Heinz ...,Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig
...,...,...,...,...,...,...,...
1957,762,/gast/geschaefte/1331,Motion der erweiterten Justizprüfungskommissio...,Datenmigration,,Motion,abgeschlossen
1963,666,/gast/geschaefte/1325,Motion der Kommission Teilrevision Personalges...,Datenmigration,,Motion,abgeschlossen
1979,304,/gast/geschaefte/1309,Motion von Manuela Weichelt betreffend HIV-Prä...,Datenmigration,,Motion,abgeschlossen
1980,282,/gast/geschaefte/1712,Motion von Christoph Hohler betreffend Radstre...,Frist erstreckt,Baudirektion,Motion,hängig


In [7]:
df_v=df_v.reset_index()#Ich will überall nun denselben Index. deshalb nur soviel, wie ich Zeilen habe
df_v.to_csv("KRFrames/df_v_Hauptseiten")

### 1.1.1 Eine Liste aus allen Geschäfts-URLs machen (um später auf diese zuzugreiffen)

In [8]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_v["url"]:
    url_liste_ganz.append(url_anfang_original+element)
#diese werden zusammengefügt aus dem Anfang der Adresse sowie dem Geschäfts-URL

In [9]:
len(url_liste_ganz)

1227

### Die einzelnen Geschäfte aufrufen und Infos rausholen

# -> Einschub: ich will speichern und mit diesen Daten dann arbeiten.

In [11]:
#habe nun alle Seiten der Geschäfte auf meinem Rechner gespeichert. 
number = 0
for page in url_liste_ganz:
    page_content = requests.get(page)
    page_content = page_content.text
    with open("KRhtml/Geschaeft"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1
    #Es sind nun exakt so viele Seiten auf meinem Rechner, wie das DataFrame oben Zeilen anzeigt

# <-Einschub-Ende. Ich habe es geschafft, das zu speichern. 

In [17]:
#nun muss ich den Code ändern, sodass ich nicht mehr online zugreiffe sondern über meinen Ordner.
#hier nun die Liste der Zugriffe.
geschaefts_liste=[]

for seite_g in range(0,1226):
    gesch= "KRhtml/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [18]:
len(geschaefts_liste)

1226

In [19]:
geschaefts_liste[-10:]

['KRhtml/Geschaeft1216.html',
 'KRhtml/Geschaeft1217.html',
 'KRhtml/Geschaeft1218.html',
 'KRhtml/Geschaeft1219.html',
 'KRhtml/Geschaeft1220.html',
 'KRhtml/Geschaeft1221.html',
 'KRhtml/Geschaeft1222.html',
 'KRhtml/Geschaeft1223.html',
 'KRhtml/Geschaeft1224.html',
 'KRhtml/Geschaeft1225.html']

# 2. Zugang zu den PDFs

Ich musste rausfinden, bei welchen Files ich nicht auf die Nummer 1 zugreiffen kann. 
*Siehe File: "Falsche Files rausfinden - code anpassen"* 

In [20]:
#hier  habe ich mich entschieden, auf den Zugriff der Namen zu verzichten, und dies einheitlich über die PDFs zu lösen.
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
eingang_list=[]


for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument.
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
   
   
     #Da es zum Teil Referenzen hat und dadurch ein anderer Ort abgegriffen werden muss, muss ich varieren.
    
    if td_g_list[-1].find_all("a")[0].text== "1": #Ich brauche immer das Dokument 1. 
        url_v_pdf = td_g_list[-1].find("a")["href"]
    
    elif td_g_list[-1].find_all("a")[0].text== "2": #bei älteren GEschäften sind es Bericht&Anträge
        url_v_pdf = td_g_list[-1].find("a")["href"]   
    else: 
        td_g_list[-1].find_all("a")[0].text== "3"
        url_v_pdf = td_g_list[-1].find("a")["href"]
   
            
    
    minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf,}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

In [21]:
df_g=pd.DataFrame(eingang_list)

In [22]:
#Das Datum muss noch als solches lesbar gemacht werden
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum
df_g["VorlageNR"] =df_g["VorlageNR"].astype(int)

In [23]:
df_g.to_csv("KRFrames/df_g_LinkszuPDF")

In [2]:
df_g=pd.read_csv("KRFrames/df_g_LinkszuPDF")

In [6]:
df_gast=df_g[df_g["Link Vorstoss-PDF"].str.contains("gast")]

In [8]:
df_gast.sort_values("Einreichedatum").tail(50)# Also, es haben alle bis auf ganz wenige Daten.

,Unnamed: 0,VorlageNR,Einreichedatum,Link Vorstoss-PDF
367,367,2611,2015-01-29,/gast/geschaefte/1597
368,368,2509,2015-01-29,/gast/geschaefte/1495
360,360,2737,2015-03-09,/gast/geschaefte/1727
358,358,2407,2015-03-10,/gast/geschaefte/1388
357,357,2746,2015-03-13,/gast/geschaefte/1735
355,355,2652,2015-03-17,/gast/geschaefte/1639
347,347,2129,2015-05-01,/gast/geschaefte/381
346,346,2911,2015-05-08,/gast/geschaefte/1899
345,345,2477,2015-05-11,/gast/geschaefte/1465
343,343,2129,2015-05-27,/gast/geschaefte/381


# 3. PDFs der Vorstösse runterladen

In [25]:
for link in tqdm(df_g["Link Vorstoss-PDF"]):
    r = requests.get("https://kr-geschaefte.zug.ch"+link, stream =True) #Das Stream braucht es, um mit dem Zip umgehen zu koennen
    name = link.split("/")[-1] #Der Computer kann mit "/" in einem Namen nicht umgehen, deshalb nehme ich die hier raus, und nehme nur den letzten Teil des NAmens.
    with open("KRGeschPDF/"+name, "wb") as f:
        f.write(r.content)

100%|██████████| 1226/1226 [03:16<00:00,  6.24it/s]


# 4. PDFs sind auf dem Rechner, nun lesbar machen und !Regex Baby!

In [2]:
#nun will ich aus jedem PDF zum Einen die VorlagenNR rausziehen sowie Namen und Datum hier die Definition
# ! Danke dem |$ am Schluss gibt es keinen Index-Error, wenn er kein Resultat findet.
regex_vorlage= r"VORLAGE.NR\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_datum = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [3]:
#Lösung online gefunden um aus dem PDF-Ordner eine Liste mit den Links herzustellen.
pdf_files_list= [f for f in listdir("KRGeschPDF") if isfile(join("KRGeschPDF", f))]

In [4]:
len(pdf_files_list) #Ich habe ja die PDFs ignoriert, zu welchen ich auf Grund der Referenz keine PDF-Links hatte.

1099

In [5]:
# !!!!!! ACHTUNG: zuerst mit einzelen testen. Möglicherweise findet er die Files nicht. 
dat_pdf_list=[]
for dok in pdf_files_list:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_name = dok
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_ein= re.findall(regex_datum, front, re.IGNORECASE)[0] #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    
    minidict_pdf_dat={"Einreichedatum": dat_p_ein, "VorlageNR":vorl_p_nr, "Filename":pdf_name}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [6]:
df_pr=pd.DataFrame(dat_pdf_list)

In [7]:
df_pr.sort_values("Einreichedatum") #Stichproben haben gezeigt, dass die fehlerhaften Datumsangaben in GEschäftern sind,
# Welche selber meist schon ein Datum haben. 

,Einreichedatum,VorlageNR,Filename
670,,,2930-1-16005_Sicherheit-Schulweg.pdf
640,,,2867-1-15770_Einsatzkoordination.pdf
647,,,2884-1-15816_Zahlungen-Kantone.pdf
271,,,14235_2216_1_Krankenkassenpr%C3%A4mien.pdf
649,,,2892-1-15881_KA-Sportf%C3%B6rderung.pdf
...,...,...,...
763,9. MÄRZ 2006,1420,pdoc_1219_1.pdf
827,9. MÄRZ 2007,1518,pdoc_1682_1.pdf
322,9. November 2013,,14510_2318_1_Weiterbildungen.pdf
164,9. September 2010,,13532_1967_1_RGPK.pdf


In [8]:
df_pr.to_csv("KRFrames/INFOSausPDF")

In [6]:
df_pr=pd.read_csv("KRFrames/INFOSausPDF")
df_pr

,Unnamed: 0,Einreichedatum,VorlageNR,Filename
0,0,17. SEPTEMBER 2007,1584.0,12487_1584.1_motion.pdf
1,1,18. SEPTEMBER 2007,1589.0,12493_1589_1_motion.pdf
2,2,6. DEZEMBER 2007,1616.0,12562_1616_1_motion.pdf
3,3,31. JANUAR 2008,1635.0,12611_1635_1motion.pdf
4,4,21. FEBRUAR 2008,1644.0,12634_1644_1_interpellation.pdf
...,...,...,...,...
1094,1094,4. AUGUST 2005,1362.0,pdoc_972_1.pdf
1095,1095,12. AUGUST 2005,1363.0,pdoc_977_1.pdf
1096,1096,23. JUNI 2005,1364.0,pdoc_978_1.pdf
1097,1097,26. AUGUST 2005,1365.0,pdoc_984_1.pdf


### 4.0.1 Fehlende Datum-Angaben (File 04) reinnehmen

In [37]:
df_fp=pd.read_csv("KRFrames/fehlende_Daten")

In [38]:
df_pr.rename(columns={"Filename": "Link Vorstoss-PDF"}, inplace=True)
df_fp.rename(columns={"Einreichedatum3": "Einreichedatum"}, inplace=True) #Spalte gleich benennen wie im anderen Frame

In [39]:
df_p1=pd.merge(df_pr,df_fp, how="outer" , on= "Link Vorstoss-PDF")

In [40]:
spalte_vereint=df_p1.Einreichedatum_x.combine_first(df_p1.Einreichedatum_y)

In [41]:
df_p1["Einreichedatum"]= spalte_vereint

In [42]:
df_p=df_p1[["VorlageNR_x", "Einreichedatum", "Link Vorstoss-PDF"]]

In [44]:
df_p.rename(columns={"VorlageNR_x":"VorlageNR"}, inplace=True)
df_p.to_csv("KRFrames/alle_pdf_daten")

In [34]:
df_p.dtypes

VorlageNR            object
Einreichedatum       object
Link Vorstoss-PDF    object
dtype: object

## 4.1 Zeit in Datetime umwandeln
schwierigkeit: wie bringe ich das deutsche Zeitformat in englisches. Mache ich es falsch oder gibts wieder probleme mit den Grossbuchstaben der Zeitformate? Hatten wir windowsuser auch schon.

In [46]:
df_p=pd.read_csv("KRFrames/alle_pdf_daten")

In [47]:
locale.setlocale(locale.LC_ALL, 'de_DE') # hier die Sprachwahl
df_p["Einreichedatum"] = pd.to_datetime(df_p["Einreichedatum"], format= "%d. %B %Y", errors='coerce') #Angabe des Formats

In [48]:
 df_p #Halleluia:-) habs doch noch geschafft. Und die wenigen, die falsch formiert sind, werden nicht auffallen.

,Unnamed: 0,VorlageNR,Einreichedatum,Link Vorstoss-PDF
0,0,1584.0,2007-09-17,12487_1584.1_motion.pdf
1,1,1589.0,2007-09-18,12493_1589_1_motion.pdf
2,2,1616.0,2007-12-06,12562_1616_1_motion.pdf
3,3,1635.0,2008-01-31,12611_1635_1motion.pdf
4,4,1644.0,2008-02-21,12634_1644_1_interpellation.pdf
...,...,...,...,...
1094,1094,1362.0,2005-08-04,pdoc_972_1.pdf
1095,1095,1363.0,2005-08-12,pdoc_977_1.pdf
1096,1096,1364.0,2005-06-23,pdoc_978_1.pdf
1097,1097,1365.0,2005-08-26,pdoc_984_1.pdf


In [49]:
df_p.to_csv("KRFrames/KonvertierteDatenformate")

In [50]:
df_p=pd.read_csv("KRFrames/KonvertierteDatenformate")

In [51]:
#Nun noch ohne.0 hinten bei der VorlageNR. Ah es ist ein Float. Kann es also zu int oder str machen
df_p["VorlageNR"]=df_p["VorlageNR"].astype(str)

In [54]:
def splitting(elem):
    elem = elem.split('.')[0].strip()
    return elem
df_p["VorlageNR"]=df_p["VorlageNR"].apply(splitting) # So, nun sind die .0 weg. Mal schauen. Das String-Format reicht wohl

In [55]:
df_p

,Unnamed: 0,Unnamed: 0.1,VorlageNR,Einreichedatum,Link Vorstoss-PDF
0,0,0,1584,2007-09-17,12487_1584.1_motion.pdf
1,1,1,1589,2007-09-18,12493_1589_1_motion.pdf
2,2,2,1616,2007-12-06,12562_1616_1_motion.pdf
3,3,3,1635,2008-01-31,12611_1635_1motion.pdf
4,4,4,1644,2008-02-21,12634_1644_1_interpellation.pdf
...,...,...,...,...,...
1094,1094,1094,1362,2005-08-04,pdoc_972_1.pdf
1095,1095,1095,1363,2005-08-12,pdoc_977_1.pdf
1096,1096,1096,1364,2005-06-23,pdoc_978_1.pdf
1097,1097,1097,1365,2005-08-26,pdoc_984_1.pdf


In [56]:
df_ps=df_p[["VorlageNR", "Einreichedatum", "Link Vorstoss-PDF"]]#nun noch hübschen.

In [57]:
df_ps.to_csv("KRFrames/df_p_Datum") # und speichern

In [31]:
df_ps=pd.read_csv("KRFrames/df_p_Datum")
df_ps

,Unnamed: 0,Einreichedatum,VorlageNR,Link Vorstoss-PDF
0,0,2007-09-17,1584.0,12487_1584.1_motion.pdf
1,1,2007-09-18,1589.0,12493_1589_1_motion.pdf
2,2,2007-12-06,1616.0,12562_1616_1_motion.pdf
3,3,2008-01-31,1635.0,12611_1635_1motion.pdf
4,4,2008-02-21,1644.0,12634_1644_1_interpellation.pdf
...,...,...,...,...
1088,1088,2005-08-04,1362.0,pdoc_972_1.pdf
1089,1089,2005-08-12,1363.0,pdoc_977_1.pdf
1090,1090,2005-06-23,1364.0,pdoc_978_1.pdf
1091,1091,2005-08-26,1365.0,pdoc_984_1.pdf


## 4.2 Ein Dataframe machen, mit den PDF-Dokumentennamen
- splitting funktion schreiben
- auf Spalte anwenden
- neu abspeichern 

In [58]:
def splittpdf(elem):
    elem = elem.split('/')[-1].strip()
    return elem

In [61]:
df_g["Link Vorstoss-PDF"]=df_g["Link Vorstoss-PDF"].apply(splittpdf)

In [24]:
df_g.to_csv("KRFrames/df_g_Datum_mit_PDF")

## 4.3 Nun zusammenfügen der beiden Daten-Dataframes anhand der PDF-Namen

In [62]:
df_g1=pd.read_csv("KRFrames/df_g_Datum_mit_PDF")
df_p1=pd.read_csv("KRFrames/df_p_Datum")

In [63]:
df_dat_max=pd.merge(df_g1, df_p1, how="left",on= "Link Vorstoss-PDF")

In [64]:
df_dat_max.shape

(1226, 8)

In [65]:
df_dat_max.sort_values("Einreichedatum_x").tail(100)  #So, jetzt sind zwar beide zusammen, aber noch nicht vereint
#Jetzt noch irgendwie die beiden Einreichedaten miteinander ableichen/ergängen.

,Unnamed: 0_x,Unnamed: 0.1,VorlageNR_x,Einreichedatum_x,Link Vorstoss-PDF,Unnamed: 0_y,VorlageNR_y,Einreichedatum_y
1125,1125,1125,1169,NaN,pdoc_535_1.pdf,983.0,1169.0,2003-09-17
1126,1126,1126,1166,NaN,pdoc_526_1.pdf,982.0,1166.0,2003-09-08
1127,1127,1127,1165,NaN,pdoc_525_1.pdf,981.0,1165.0,2003-09-08
1128,1128,1128,1164,NaN,pdoc_524_1.pdf,980.0,1164.0,2003-09-08
1129,1129,1129,1159,NaN,pdoc_498_1.pdf,978.0,1159.0,2003-09-01
...,...,...,...,...,...,...,...,...
1220,1220,1220,810,NaN,pdoc_62_1.pdf,1005.0,810.0,2002-08-13
1221,1221,1221,801,NaN,pdoc_1440_1.pdf,794.0,801.0,2006-08-22
1222,1222,1222,762,NaN,pdoc_1047_1.pdf,741.0,NaN,2005-10-31
1223,1223,1223,666,NaN,pdoc_55_1.pdf,987.0,666.0,2002-06-06


### 4.3.1 die Datenspalten zusammenbekommen.

In [66]:
#hier füge ich sie zusammen
df_dat_zus=df_dat_max.Einreichedatum_x.combine_first(df_dat_max.Einreichedatum_y)

In [67]:
df_dat_max['Einreichedatum'] = df_dat_zus # hier füge ich die neu enstandene "Series" dem Dataframe an

In [68]:
df_datum=df_dat_max[["Unnamed: 0_x","VorlageNR_x", "Einreichedatum", "Link Vorstoss-PDF"]]

In [69]:
df_datum.shape

(1226, 4)

In [77]:
df_datum.sort_values("Einreichedatum").tail(14)
                                             

,Unnamed: 0_x,VorlageNR_x,Einreichedatum,Link Vorstoss-PDF
0,0,3031,2019-11-26,3031-1-16191_Neutralitaet_Schulen.pdf
494,494,2299,NaN,14464_2299_1_Kinderbetreuung.pdf
509,509,2273,NaN,14391_2273_1_Halbanschluss.pdf
568,568,2175,NaN,14140_2175_1_Auftr%C3%A4ge.pdf
583,583,2718,NaN,1707
639,639,2035,NaN,13730_2035_1_Atomenergie.pdf
645,645,2526,NaN,1512
756,756,1817,NaN,13075_1817_1_Strassenbau.pdf
834,834,1710,NaN,12812.pdf
839,839,2194,NaN,446


In [78]:
df_datum.to_csv("KRFrames/df_datum_ALLE")

# 5. Alle Dataframes INKL UNTERSEITEN zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [2]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat_all=pd.read_csv("KRFrames/df_datum_ALLE")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Frame_Namen_Partei_endversion")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



In [3]:
df_dat_all.sort_values("Einreichedatum").tail(20)

,Unnamed: 0,Unnamed: 0_x,VorlageNR_x,Einreichedatum,Link Vorstoss-PDF
6,6,6,3021,2019-10-18,3021-1-16169_Polizei.pdf
5,5,5,3023,2019-10-31,3023-1-16177_Dreckschleudern.pdf
4,4,4,3024,2019-11-06,3024-1-16179_Stadtbahn.pdf
3,3,3,3026,2019-11-15,3026-1-16186_Airbnb.pdf
2,2,2,3029,2019-11-24,3029-1-16189_Glencore.pdf
1,1,1,3030,2019-11-26,3030-1-16190_Gewaltentrennung.pdf
0,0,0,3031,2019-11-26,3031-1-16191_Neutralitaet_Schulen.pdf
494,494,494,2299,NaN,14464_2299_1_Kinderbetreuung.pdf
509,509,509,2273,NaN,14391_2273_1_Halbanschluss.pdf
568,568,568,2175,NaN,14140_2175_1_Auftr%C3%A4ge.pdf


In [4]:
df_dat_all=df_dat_all.rename(columns={"VorlageNR_x": "VorlageNR"})

In [21]:
#  zuerst vorputzen
df_v1 = df_v[["VorlageNR", "Geschäft", "Zuständigkeit", "Art des Geschäfts", "Stand", "url"]].copy()
df_v1.set_index("VorlageNR", inplace=True)

In [22]:
df_v1

,Geschäft,Zuständigkeit,Art des Geschäfts,Stand,url
VorlageNR,,,,,
3031,Interpellation der SVP-Fraktion betreffend Sic...,Direktion für Bildung und Kultur,Interpellation,Einreichung,/gast/geschaefte/2018
3030,Motion der SVP-Fraktion betreffend Stärkung de...,NaN,Motion,Einreichung,/gast/geschaefte/2017
3029,Interpellation der Fraktion Alternative - die ...,Volkswirtschaftsdirektion,Interpellation,Einreichung,/gast/geschaefte/2016
3026,Interpellation von Markus Spörri und Thomas Ga...,Baudirektion,Interpellation,Einreichung,/gast/geschaefte/2013
3024,Interpellation von Claus Soltermann und Heinz ...,Baudirektion,Interpellation,Überweisung an Regierungsrat,/gast/geschaefte/2012
...,...,...,...,...,...
762,Motion der erweiterten Justizprüfungskommissio...,NaN,Motion,Datenmigration,/gast/geschaefte/1331
666,Motion der Kommission Teilrevision Personalges...,NaN,Motion,Datenmigration,/gast/geschaefte/1325
304,Motion von Manuela Weichelt betreffend HIV-Prä...,NaN,Motion,Datenmigration,/gast/geschaefte/1309


In [23]:
# Nun zusammenfügen
df_alles=pd.merge(df_v1, df_dat_all, how="inner",on= "VorlageNR")

In [24]:
df_alles.sort_values("Einreichedatum").tail(14) #ok, die letzten 12 haben kein Datum. 
#Die Prüfung hat jedoch ergeben, dass diese über alle Jahre verteilt sind und noch 3 unparteiische dabei sind.

,VorlageNR,Geschäft,Zuständigkeit,Art des Geschäfts,Stand,url,Unnamed: 0,Unnamed: 0_x,Einreichedatum,Link Vorstoss-PDF
1,3030,Motion der SVP-Fraktion betreffend Stärkung de...,NaN,Motion,Einreichung,/gast/geschaefte/2017,1,1,2019-11-26,3030-1-16190_Gewaltentrennung.pdf
0,3031,Interpellation der SVP-Fraktion betreffend Sic...,Direktion für Bildung und Kultur,Interpellation,Einreichung,/gast/geschaefte/2018,0,0,2019-11-26,3031-1-16191_Neutralitaet_Schulen.pdf
202,2718,Interpellation der Fraktion Alternative - die ...,Volkswirtschaftsdirektion,Interpellation,Kenntnisnahme,/gast/geschaefte/1707,583,583,NaN,1707
203,2718,Interpellation der Fraktion Alternative - die ...,Volkswirtschaftsdirektion,Interpellation,Kenntnisnahme,/gast/geschaefte/1707,1013,1013,NaN,1707
317,2526,Motion von Anna Bieri und Laura Dittli betreff...,Direktion für Bildung und Kultur,Motion,Erledigterklärung,/gast/geschaefte/1512,645,645,NaN,1512
452,2299,Kleine Anfrage von Stefan Gisler zu Aussagen d...,NaN,Kleine Anfrage,Datenmigration,/gast/geschaefte/556,494,494,NaN,14464_2299_1_Kinderbetreuung.pdf
465,2273,Postulat von Florian Weber und Franz Hürlimann...,NaN,Postulat,Datenmigration,/gast/geschaefte/525,509,509,NaN,14391_2273_1_Halbanschluss.pdf
515,2175,Interpellation von Thomas Aeschi betreffend Üb...,NaN,Interpellation,Datenmigration,/gast/geschaefte/427,568,568,NaN,14140_2175_1_Auftr%C3%A4ge.pdf
578,2035,Postulat der Alternativen Grünen Fraktion und ...,NaN,Postulat,Datenmigration,/gast/geschaefte/290,639,639,NaN,13730_2035_1_Atomenergie.pdf
633,1922,Postulat der erweiterten Jusitzprüfungskommiss...,Finanzdirektion,Postulat,Erledigterklärung,/gast/geschaefte/183,849,849,NaN,183


In [25]:
df_alles.to_csv("KRFrames/df_alles_ohne_Namen")

In [26]:
df_np

,Unnamed: 0,Namenganz,P,Unnamed: 0.1,Unnamed: 0.1.1,VorlageNR
0,0,Peter Rust,CVP,0.0,NaN,1432
1,1,Peter Rust,CVP,1.0,NaN,1388
2,2,Peter Rust,CVP,2.0,NaN,1383
3,3,Peter Rust,CVP,3.0,NaN,1324
4,4,Peter Rust,CVP,4.0,NaN,991
...,...,...,...,...,...,...
1917,339,Alternative,ALG,NaN,NaN,1932
1918,358,Alternative,ALG,NaN,NaN,1694
1919,359,Alternative,ALG,NaN,NaN,1679
1920,363,Alternative,ALG,NaN,NaN,1608


In [27]:
df_alles_ganz=pd.merge(df_alles, df_np, how="inner", on="VorlageNR")
df_alles_ganz.shape

(1809, 15)

In [28]:
df_alles_ganz.sort_values("P").tail(35)

,VorlageNR,Geschäft,Zuständigkeit,Art des Geschäfts,Stand,url,Unnamed: 0_x,Unnamed: 0_x,Einreichedatum,Link Vorstoss-PDF,Unnamed: 0_y,Namenganz,P,Unnamed: 0.1,Unnamed: 0.1.1
1470,1468,"Interpellation von Monika Barmet, Karl Künzle,...",NaN,Interpellation,Datenmigration,/gast/geschaefte/1052,973,973,2006-08-16,pdoc_1413_1.pdf,484,Karl Nussbaumer,SVP,484.0,NaN
350,2742,Motion der SVP-Fraktion betreffend transparent...,Obergericht,Motion,Erledigterklärung,/gast/geschaefte/1732,191,191,2017-05-03,2742-1-15438-Landesverweisungen.pdf,1387,SVP-Fraktion,SVP,1387.0,NaN
349,2743,Motion der SVP-Fraktion betreffend Aufhebung d...,Direktion für Bildung und Kultur,Motion,Nicht-Erheblicherklärung,/gast/geschaefte/1733,190,190,2017-05-04,2743-1-15441_Schulpsychologischer_Dienst.pdf,1386,SVP-Fraktion,SVP,1386.0,NaN
0,3031,Interpellation der SVP-Fraktion betreffend Sic...,Direktion für Bildung und Kultur,Interpellation,Einreichung,/gast/geschaefte/2018,0,0,2019-11-26,3031-1-16191_Neutralitaet_Schulen.pdf,1444,SVP-Fraktion,SVP,1444.0,NaN
695,2379,"Interpellation von Martin Stuber, Philip C. Br...",Sicherheitsdirektion,Interpellation,Kenntnisnahme,/gast/geschaefte/1371,435,435,2014-04-10,14659_2379_1_Polycom.pdf,908,Philip C. Brunner,SVP,908.0,NaN
318,2769,"Motion von Jürg Messmer, Andreas Hostettler un...",Direktion für Bildung und Kultur,Motion,Nicht-Erheblicherklärung,/gast/geschaefte/1759,171,171,2017-08-02,2769-1-15519__Franz%C3%B6sisch.pdf,886,Philip C. Brunner,SVP,886.0,NaN
713,2356,"Interpellation von Manuel Brandenberg, Philip ...",Volkswirtschaftsdirektion,Interpellation,Kenntnisnahme,/gast/geschaefte/605,452,452,2014-02-03,14576_2356_1_Einwanderung.pdf,863,Manuel Brandenberg,SVP,863.0,NaN
319,2769,"Motion von Jürg Messmer, Andreas Hostettler un...",Direktion für Bildung und Kultur,Motion,Nicht-Erheblicherklärung,/gast/geschaefte/1759,171,171,2017-08-02,2769-1-15519__Franz%C3%B6sisch.pdf,1084,Jürg Messmer,SVP,1084.0,NaN
712,2357,Interpellation von Philip C. Brunner und Jürg ...,Finanzdirektion,Interpellation,Kenntnisnahme,/gast/geschaefte/606,451,451,2014-02-05,14577_2357_1_CoGeneralsekret%C3%A4r.pdf,1088,Jürg Messmer,SVP,1088.0,NaN
1525,1378,"Motion von Hans Christen, Eusebius Spescha, Be...",NaN,Motion,Datenmigration,/gast/geschaefte/968,1014,1014,2005-10-03,pdoc_1011_1.pdf,507,Beat Stocker,SVP,507.0,NaN


In [29]:
df_tuti=df_alles_ganz[["VorlageNR","Geschäft", "Namenganz", "P", "Einreichedatum", "Stand", "Link Vorstoss-PDF","Zuständigkeit","Art des Geschäfts","url"]]

In [30]:
df_tuti1=df_tuti.drop_duplicates()
# Obwohl mir nur von 30 Geschäften das Datum fehlt, explodiert dies, wenn es auf die Namen-Tabelle kommt.
df_tuti1.shape

(1803, 10)

In [31]:
df_tuti1.sort_values("Einreichedatum").tail(30)

,VorlageNR,Geschäft,Namenganz,P,Einreichedatum,Stand,Link Vorstoss-PDF,Zuständigkeit,Art des Geschäfts,url
15,3019,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",Andreas Lustenberger,ALG,2019-10-11,Überweisung an Regierungsrat,3019-1-16167_Chancengleichheit.pdf,Direktion des Innern,Motion,/gast/geschaefte/2007
14,3020,Postulat von Luzian Franzini und Esther Haas b...,Luzian Franzini,ALG,2019-10-11,Überweisung an Regierungsrat,3020-1-16168_Racial-Profiling.pdf,Sicherheitsdirektion,Postulat,/gast/geschaefte/2008
13,3020,Postulat von Luzian Franzini und Esther Haas b...,Esther Haas,ALG,2019-10-11,Überweisung an Regierungsrat,3020-1-16168_Racial-Profiling.pdf,Sicherheitsdirektion,Postulat,/gast/geschaefte/2008
8,3021,"Interpellation von Philip C. Brunner, Barbara ...",Barbara Gysel,SP,2019-10-18,Überweisung an Regierungsrat,3021-1-16169_Polizei.pdf,Sicherheitsdirektion,Interpellation,/gast/geschaefte/2009
9,3021,"Interpellation von Philip C. Brunner, Barbara ...",Philip C. Brunner,SVP,2019-10-18,Überweisung an Regierungsrat,3021-1-16169_Polizei.pdf,Sicherheitsdirektion,Interpellation,/gast/geschaefte/2009
11,3021,"Interpellation von Philip C. Brunner, Barbara ...",Tabea Zimmermann Gibson,ALG,2019-10-18,Überweisung an Regierungsrat,3021-1-16169_Polizei.pdf,Sicherheitsdirektion,Interpellation,/gast/geschaefte/2009
12,3021,"Interpellation von Philip C. Brunner, Barbara ...",Anna Spescha,SP,2019-10-18,Überweisung an Regierungsrat,3021-1-16169_Polizei.pdf,Sicherheitsdirektion,Interpellation,/gast/geschaefte/2009
10,3021,"Interpellation von Philip C. Brunner, Barbara ...",Andreas Lustenberger,ALG,2019-10-18,Überweisung an Regierungsrat,3021-1-16169_Polizei.pdf,Sicherheitsdirektion,Interpellation,/gast/geschaefte/2009
7,3023,Interpellation der SP-Fraktion betreffend «Dre...,SP-Fraktion,SP,2019-10-31,Überweisung an Regierungsrat,3023-1-16177_Dreckschleudern.pdf,Sicherheitsdirektion,Interpellation,/gast/geschaefte/2011
6,3024,Interpellation von Claus Soltermann und Heinz ...,Heinz Achermann,CVP,2019-11-06,Überweisung an Regierungsrat,3024-1-16179_Stadtbahn.pdf,Baudirektion,Interpellation,/gast/geschaefte/2012


In [ ]:
df_tuti1.to_csv("KRFrames/df_tuti1")

### In den Namen noch die Partei hinzufügen


In [2]:
df_tuti1=pd.read_csv("KRFrames/df_tuti1")

In [5]:
df_tuti1["Namenganz"]=df_tuti1["Namenganz"]+" "+df_tuti1["P"]

In [8]:
df_tuti1.pop("Unnamed: 0")

0          0
1          1
2          2
3          3
4          4
        ... 
1798    1804
1799    1805
1800    1806
1801    1807
1802    1808
Name: Unnamed: 0, Length: 1803, dtype: int64

In [10]:
df_tuti1.to_csv("KRFrames/df_tuti_Ganzes_Frame")

# Geplottet wird in den Files 05 und 06